In [1]:
import geopandas as gpd
import pandas as pd

In [ ]:
#pd.set_option('display.max_columns', None)

In [2]:
ibb = gpd.GeoDataFrame.from_file('data/clean_ibb.geojson')
osm = gpd.GeoDataFrame.from_file('data/clean_osm.geojson')
hgm = gpd.GeoDataFrame.from_file('data/clean_hgm.geojson')

In [3]:
osm['wp'] = osm.wikipedia[osm['wikipedia'].astype(str).str.contains('en:')].str.replace(" ","_").str.replace("en:","dbr:")
osm_wp1 = osm[['fid','wp']][osm['wp'].notnull()]
osm['wp'] = ('dbr:' + osm['wikipedia_en'][osm['wikipedia_en'].notnull()].str.replace(" ","_"))
osm_wp2 = osm[['fid','wp']][osm['wp'].notnull()]
osm_wp = pd.concat([osm_wp1,osm_wp2],ignore_index=True)
dbpedia = [('idt:' + osm_wp['fid'][i] + ' owl:sameAs ' + osm_wp['wp'][i] + ' .' ) for i in range(len(osm_wp))]

In [4]:
osm['wd'] = ('<https://www.wikidata.org/wiki/' + osm['wikidata'][osm['wikidata'].notnull()] +'>')
osm_wd = osm[['fid','wd']][osm['wd'].notnull()].reset_index(drop=True)
wikidata = [('idt:' + osm_wd['fid'][i] + ' owl:sameAs ' + osm_wd['wd'][i] + ' .' ) for i in range(len(osm_wd))]

In [5]:
ibbDbpedia = [('idt:' + ibb['fid'][i] + ' dct:subject dbc:Fatih .') for i in range(len(ibb))]

In [6]:
yago = [('idt:' + ibb['fid'][i] + ' schema:containedInPlace yago:Fatih .') for i in range(len(ibb))]

In [ ]:
links = dbpedia + wikidata + ibbDbpedia + yago

with open('data/links.ttl', 'w') as output:
        for i in range(len(links)):
            output.write('\n' + links[i] + '\n')